# AD Privileged Users or Groups Reconnaissance
Detect priv users or groups recon based on 4661 eventid and known privileged users or groups SIDs

## Rule Content
```
- title: AD Privileged Users or Groups Reconnaissance
  id: 35ba1d85-724d-42a3-889f-2e2362bcaf23
  description: Detect priv users or groups recon based on 4661 eventid and known privileged
    users or groups SIDs
  references:
  - https://blog.menasec.net/2019/02/threat-hunting-5-detecting-enumeration.html
  tags:
  - attack.discovery
  - attack.t1087
  status: experimental
  author: Samir Bousseaden
  logsource:
    product: windows
    service: security
    definition: 'Requirements: enable Object Access SAM on your Domain Controllers'
    category: null
  detection:
    selection:
      EventID: 4661
      ObjectType:
      - SAM_USER
      - SAM_GROUP
      ObjectName:
      - '*-512'
      - '*-502'
      - '*-500'
      - '*-505'
      - '*-519'
      - '*-520'
      - '*-544'
      - '*-551'
      - '*-555'
      - '*admin*'
    condition: selection
  falsepositives:
  - if source account name is not an admin then its super suspicious
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-security-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"4661" AND object_type:("SAM_USER" OR "SAM_GROUP") AND object_name.keyword:(*\-512 OR *\-502 OR *\-500 OR *\-505 OR *\-519 OR *\-520 OR *\-544 OR *\-551 OR *\-555 OR *admin*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()